In [460]:
import sys
!{sys.executable} -m pip install tabulate

In [462]:
import requests
import xmltodict
from datetime import datetime
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import pprint
from ast import literal_eval
from tabulate import tabulate
pp = pprint.PrettyPrinter(indent=2)

In [3]:
players = pd.read_csv("players2019.csv")

In [340]:
trades = pd.read_csv("trades2019.csv", converters={
    "trader_player_keys_received": literal_eval,
    "trader_player_names_received": literal_eval,
    "tradee_player_keys_received": literal_eval,
    "tradee_player_names_received": literal_eval})

In [279]:
teams = pd.read_csv("teams2019.csv")
teams = teams.set_index("name")
teams

,team_key,number_of_moves,number_of_trades,clinched_playoffs,draft_grade,manager_nickname,division_id
name,,,,,,,
Professor Teabag,390.l.99174.t.1,33,2,True,C+,Miccyd,2
Champ,390.l.99174.t.2,27,0,True,B,Simon Kim,2
Bottom Bitches,390.l.99174.t.3,31,2,False,B,Justin B,1
Hareem Kunt,390.l.99174.t.4,39,1,True,C,Paul Masini,2
Salmon Sisters,390.l.99174.t.5,40,4,True,B,Matt M,2
Hike School,390.l.99174.t.6,21,2,True,C,J Remedy,1
1503 Sequoia Trail,390.l.99174.t.7,26,0,False,A,Chris A,1
Burrito House,390.l.99174.t.8,42,1,False,B+,Greg A,1
Pepper Brooks,390.l.99174.t.9,37,1,False,B+,Sam,2


In [280]:
matchups = pd.read_csv("matchups2019.csv")

In [7]:
matchups.head()

,week,is_playoffs,is_consolation,team_1_key,team_1_name,team_1_points,team_1_projected_points,is_bye,team_2_key,team_2_name,team_2_points,team_2_projected_points
0,1,False,False,390.l.99174.t.1,Professor Teabag,184.26,149.72,False,390.l.99174.t.9,Pepper Brooks,135.74,142.64
1,2,False,False,390.l.99174.t.1,Professor Teabag,121.24,152.38,False,390.l.99174.t.2,Champ,114.98,139.52
2,3,False,False,390.l.99174.t.1,Professor Teabag,168.72,150.28,False,390.l.99174.t.3,Bottom Bitches,117.18,147.83
3,4,False,False,390.l.99174.t.1,Professor Teabag,115.72,147.02,False,390.l.99174.t.6,Hike School,160.61,136.25
4,5,False,False,390.l.99174.t.1,Professor Teabag,111.33,142.75,False,390.l.99174.t.4,Hareem Kunt,132.18,140.26


In [70]:
positions = ['QB', 'QB', 'WR', 'WR','WR', 'RB', 'RB', 'TE', 'WR/RB/TE', 'K', 'DEF']
def optimal_player_points(players):
    points = 0.0
    player_dicts = []
    for index,item in players.iterrows():
        player_dicts.append({
            "position": item["position"],
            "points": item["points"]
        })
    for position in positions:
        allowed_positions = position.split("/")
        max_index = -1
        max_points = 0
        for i in range(0, len(player_dicts)):
            if player_dicts[i]["position"] in allowed_positions and player_dicts[i]["points"] > max_points:
                max_index = i
                max_points = player_dicts[i]["points"]
        if max_points > 0:
            points += max_points
            del player_dicts[max_index]
    return points

In [71]:
groupByTeamByWeek = players.groupby(['team_name', 'week'])
optimal = pd.DataFrame({'optimal' : groupByTeamByWeek.apply(optimal_player_points)}).reset_index()

In [126]:
mergeResultDf = matchups.merge(optimal, how="right", left_on=["team_1_name", "week"], right_on=["team_name", "week"])
mergeResultDf = mergeResultDf.drop(columns=["team_1_name"])
mergeResultDf = mergeResultDf.rename(columns={"optimal": "team_1_optimal", "team_name": "team_1_name"})
mergeResultDf = mergeResultDf.merge(optimal, how="left", left_on=["team_2_name", "week"], right_on=["team_name", "week"])
mergeResultDf = mergeResultDf.drop(columns=["team_name"])
mergeResultDf = mergeResultDf.rename(columns={"optimal": "team_2_optimal"})

In [127]:
mergeResultDf

,week,is_playoffs,is_consolation,team_1_key,team_1_points,team_1_projected_points,is_bye,team_2_key,team_2_name,team_2_points,team_2_projected_points,team_1_name,team_1_optimal,team_2_optimal
0,1,False,False,390.l.99174.t.1,184.26,149.72,False,390.l.99174.t.9,Pepper Brooks,135.74,142.64,Professor Teabag,206.87,184.14
1,2,False,False,390.l.99174.t.1,121.24,152.38,False,390.l.99174.t.2,Champ,114.98,139.52,Professor Teabag,121.24,147.18
2,3,False,False,390.l.99174.t.1,168.72,150.28,False,390.l.99174.t.3,Bottom Bitches,117.18,147.83,Professor Teabag,198.53,122.58
3,4,False,False,390.l.99174.t.1,115.72,147.02,False,390.l.99174.t.6,Hike School,160.61,136.25,Professor Teabag,122.32,172.55
4,5,False,False,390.l.99174.t.1,111.33,142.75,False,390.l.99174.t.4,Hareem Kunt,132.18,140.26,Professor Teabag,140.90,142.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Champ,151.14,NaN
156,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hareem Kunt,155.38,NaN
157,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hike School,160.68,NaN
158,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pepper Brooks,130.86,NaN


In [302]:
group_by_team = mergeResultDf[(mergeResultDf["week"] < 14)].groupby("team_1_name")
recordDf = pd.DataFrame({'wins' : group_by_team.apply(lambda x: len(x[x["team_1_points"] > x["team_2_points"]].index)),
                       'losses' : group_by_team.apply(lambda x: len(x[x["team_1_points"] < x["team_2_points"]].index)),
                       'regular_season_points' : group_by_team["team_1_points"].sum(),
                       'optimal_wins': group_by_team.apply(lambda x: len(x[x["team_1_optimal"] > x["team_2_optimal"]].index)),
                       'optimal_losses': group_by_team.apply(lambda x: len(x[x["team_1_optimal"] < x["team_2_optimal"]].index)),
                       'optimal_regular_season_points' : group_by_team["team_1_optimal"].sum()})
recordDf = recordDf.join(teams["division_id"])

In [303]:
def get_ranks(record_df, name_column, wins_column, points_column, pin_top_of_division, rank_column_name):
    sortedRecords = record_df.sort_values([wins_column, points_column], ascending=False).groupby('division_id').head(1)
    remainingRecords = record_df.loc[~record_df.index.isin(sortedRecords.index)]
    sortedRecords = sortedRecords.append(remainingRecords.sort_values([wins_column, points_column], ascending=False)).reset_index()
    return pd.DataFrame(sortedRecords.index + 1, columns=[rank_column_name], index=sortedRecords[name_column])

In [304]:
optimalRanks = get_ranks(recordDf, "team_1_name", "optimal_wins", "optimal_regular_season_points", True, "optimal_rank")
ranks = get_ranks(recordDf, "team_1_name", "wins", "regular_season_points", True, "rank")
recordDf = recordDf.join(optimalRanks)
recordDf = recordDf.join(ranks)
recordDf

,wins,losses,regular_season_points,optimal_wins,optimal_losses,optimal_regular_season_points,division_id,optimal_rank,rank
team_1_name,,,,,,,,,
1503 Sequoia Trail,4,9,1688.52,5,8,1882.32,1,8,8
Bottom Bitches,4,9,1621.31,5,8,1823.81,1,9,9
Bulgogi,7,6,1878.42,5,8,2024.88,1,6,4
Burrito House,3,10,1580.91,2,11,1745.24,1,10,10
Champ,6,7,1743.66,9,4,1926.59,2,4,6
Hareem Kunt,7,6,1781.85,9,4,1962.88,2,3,5
Hike School,8,5,1818.43,7,6,2079.66,1,2,2
Pepper Brooks,5,8,1636.13,5,8,1885.77,2,7,7
Professor Teabag,11,2,1982.57,10,3,2208.86,2,1,1


In [154]:
# game 1, game 2, game 3, game 4, game 5, game 6
# rank 1 is bye
# rank 2 is bye
# rank 3 plays rank 6
def check_winner(team, other, week, points_column):
    team_points = mergeResultDf[(mergeResultDf["team_1_name"] == team) & (mergeResultDf["week"] == week)][points_column].values[0]
    other_points = mergeResultDf[(mergeResultDf["team_1_name"] == other) & (mergeResultDf["week"] == week)][points_column].values[0]
    if team_points > other_points:
        return team, other
    else:
        return other, team

In [314]:
def compute_playoffs(record_df, rank_column, points_column, final_rank_column_name):
    # playoffs
    winner_qf1, loser_qf1 = check_winner(record_df[record_df[rank_column] == 4].index.values[0],
                                         record_df[record_df[rank_column] == 5].index.values[0],
                                         14,
                                         points_column)
    winner_qf2, loser_qf2 = check_winner(record_df[record_df[rank_column] == 3].index.values[0],
                                         record_df[record_df[rank_column] == 6].index.values[0],
                                         14,
                                         points_column)
    winner_sf1, loser_sf1 = check_winner(record_df[record_df[rank_column] == 1].index.values[0],
                                          winner_qf1,
                                         15,
                                         points_column)
    winner_sf2, loser_sf2 = check_winner(record_df[record_df[rank_column] == 2].index.values[0],
                                          winner_qf2,
                                         15,
                                         points_column)
    fifth, sixth = check_winner(loser_qf1,
                                loser_qf2,
                                15,
                                points_column)
    first, second = check_winner(winner_sf1,
                                 winner_sf2,
                                 16,
                                 points_column)
    third, fourth = check_winner(loser_sf1,
                                 loser_sf2,
                                 16,
                                 points_column)
    # consolation
    winner_consolation_sf1, loser_consolation_sf1 = check_winner(record_df[record_df[rank_column] == 7].index.values[0],
                                                                 record_df[record_df[rank_column] == 10].index.values[0],
                                                                 15,
                                                                 points_column)
    winner_consolation_sf2, loser_consolation_sf2 = check_winner(record_df[record_df[rank_column] == 8].index.values[0],
                                                                 record_df[record_df[rank_column] == 9].index.values[0],
                                                                 15,
                                                                 points_column)
    seventh, eighth = check_winner(winner_consolation_sf1,
                                   winner_consolation_sf2,
                                   16,
                                   points_column)
    ninth, tenth = check_winner(loser_consolation_sf1,
                                loser_consolation_sf2,
                                16,
                                points_column)
    return pd.DataFrame(range(1, 11),
                        columns=[final_rank_column_name],
                        index=[first, second, third, fourth, fifth, sixth, seventh, eighth, ninth, tenth])
    

In [316]:
playoffs = compute_playoffs(recordDf, "rank", "team_1_points", "final_rank")
optimal_playoffs = compute_playoffs(recordDf, "optimal_rank", "team_1_optimal", "optimal_final_rank")
recordDf = recordDf.join(playoffs)
recordDf = recordDf.join(optimal_playoffs)
recordDf

,wins,losses,regular_season_points,optimal_wins,optimal_losses,optimal_regular_season_points,division_id,optimal_rank,rank,final_rank,optimal_final_rank
team_1_name,,,,,,,,,,,
1503 Sequoia Trail,4,9,1688.52,5,8,1882.32,1,8,8,10,10
Bottom Bitches,4,9,1621.31,5,8,1823.81,1,9,9,7,7
Bulgogi,7,6,1878.42,5,8,2024.88,1,6,4,3,5
Burrito House,3,10,1580.91,2,11,1745.24,1,10,10,8,8
Champ,6,7,1743.66,9,4,1926.59,2,4,6,6,6
Hareem Kunt,7,6,1781.85,9,4,1962.88,2,3,5,5,2
Hike School,8,5,1818.43,7,6,2079.66,1,2,2,2,4
Pepper Brooks,5,8,1636.13,5,8,1885.77,2,7,7,9,9
Professor Teabag,11,2,1982.57,10,3,2208.86,2,1,1,1,1


In [319]:
compute_playoffs(recordDf, "rank", "team_1_points", "final_rank")

,final_rank
Professor Teabag,1
Hike School,2
Bulgogi,3
Salmon Sisters,4
Hareem Kunt,5
Champ,6
Bottom Bitches,7
Burrito House,8
Pepper Brooks,9
1503 Sequoia Trail,10


In [318]:
compute_playoffs(recordDf, "optimal_rank", "team_1_optimal", "optimal_final_rank")

,optimal_final_rank
Professor Teabag,1
Hareem Kunt,2
Salmon Sisters,3
Hike School,4
Bulgogi,5
Champ,6
Bottom Bitches,7
Burrito House,8
Pepper Brooks,9
1503 Sequoia Trail,10


In [310]:
recordDf[recordDf["rank"] == 4].index

Index(['Bulgogi'], dtype='object', name='team_1_name')

Remap roster to "reverse" trade. then compute everything again.

In [342]:
trade = trades.head(1)
trade

,trader_team_key,trader_team_name,tradee_team_key,tradee_team_name,trader_player_keys_received,trader_player_names_received,tradee_player_keys_received,tradee_player_names_received,trader_picks_received,tradee_picks_received,week_enacted,date
0,390.l.99174.t.5,Salmon Sisters,390.l.99174.t.6,Hike School,"[390.p.29325, 390.p.31902]","[Jacoby Brissett, Darrell Henderson Jr.]","[390.p.30973, 390.p.31906]","[Sam Darnold, Devin Singletary]",[],[],9,2019-11-09 15:50:27


In [422]:
def get_players_copy_without_trade(playersDf, trade):
    playersCopy = playersDf.copy()
    playersCopy.loc[(playersCopy["player_key"].isin(trade["trader_player_keys_received"]))
               &(playersCopy["week"] > trade["week_enacted"]), "team_name"] = trade['tradee_team_name']
    playersCopy.loc[(playersCopy["player_key"].isin(trade["trader_player_keys_received"]))
               &(playersCopy["week"] > trade["week_enacted"]), "team_key"] = trade['tradee_team_key']
    playersCopy.loc[(playersCopy["player_key"].isin(trade["tradee_player_keys_received"]))
               &(playersCopy["week"] > trade["week_enacted"]), "team_name"] = trade['trader_team_name']
    playersCopy.loc[(playersCopy["player_key"].isin(trade["tradee_player_keys_received"]))
               &(playersCopy["week"] > trade["week_enacted"]), "team_key"] = trade['trader_team_key']
    return playersCopy

In [446]:
def get_records_optimal(playersDf, matchupsDf, compute_actual, optimal_rank_column_name):
    playersGroupByTeamByWeek = playersDf.groupby(['team_name', 'week'])
    optimalDf = pd.DataFrame({'optimal' : playersGroupByTeamByWeek.apply(optimal_player_points)}).reset_index()
    
    mergedMatchupsDf = matchupsDf.merge(optimalDf, how="right", left_on=["team_1_name", "week"], right_on=["team_name", "week"])
    mergedMatchupsDf = mergedMatchupsDf.drop(columns=["team_1_name"])
    mergedMatchupsDf = mergedMatchupsDf.rename(columns={"optimal": "team_1_optimal", "team_name": "team_1_name"})
    mergedMatchupsDf = mergedMatchupsDf.merge(optimal, how="left", left_on=["team_2_name", "week"], right_on=["team_name", "week"])
    mergedMatchupsDf = mergedMatchupsDf.drop(columns=["team_name"])
    mergedMatchupsDf = mergedMatchupsDf.rename(columns={"optimal": "team_2_optimal"})
    
    group_by_team = mergedMatchupsDf[(mergedMatchupsDf["week"] < 14)].groupby("team_1_name")
    recordsDf = pd.DataFrame({'wins' : group_by_team.apply(lambda x: len(x[x["team_1_points"] > x["team_2_points"]].index)),
                           'losses' : group_by_team.apply(lambda x: len(x[x["team_1_points"] < x["team_2_points"]].index)),
                           'regular_season_points' : group_by_team["team_1_points"].sum(),
                           'optimal_wins': group_by_team.apply(lambda x: len(x[x["team_1_optimal"] > x["team_2_optimal"]].index)),
                           'optimal_losses': group_by_team.apply(lambda x: len(x[x["team_1_optimal"] < x["team_2_optimal"]].index)),
                           'optimal_regular_season_points' : group_by_team["team_1_optimal"].sum()})
    recordsDf = recordsDf.join(teams["division_id"])
    optimalRanks = get_ranks(recordsDf, "team_1_name", "optimal_wins", "optimal_regular_season_points", True, "optimal_rank")
    recordsDf = recordsDf.join(optimalRanks)
    optimal_playoffs = compute_playoffs(recordsDf, "optimal_rank", "team_1_optimal", optimal_rank_column_name)
    recordsDf = recordsDf.join(optimal_playoffs)
    if compute_actual:
        ranks = get_ranks(recordsDf, "team_1_name", "wins", "regular_season_points", True, "rank")
        recordsDf = recordsDf.join(ranks)
        playoffs = compute_playoffs(recordsDf, "rank", "team_1_points", "final_rank")
        recordsDf = recordsDf.join(playoffs)
    return recordsDf

In [438]:
actualOptimal = get_records_optimal(players, matchups, compute_actual, "optmal_final_rank").sort_values("optimal_final_rank")["optimal_final_rank"]
# pp.pprint(actualOptimal)
for index, trade in trades.iterrows():
    playersDfAugmented = get_players_copy_without_trade(players, trade)
    print(f"Week {trade['week_enacted']} {trade['trader_team_name']} got {trade['trader_player_names_received']}. {trade['tradee_team_name']} got {trade['tradee_player_names_received']}.")
    pp.pprint(actualOptimal)
    pp.pprint(get_records_optimal(playersDfAugmented, matchups, "").sort_values("optimal_final_rank")["optimal_final_rank"])

Week 9 Salmon Sisters got ['Jacoby Brissett', 'Darrell Henderson Jr.']. Hike School got ['Sam Darnold', 'Devin Singletary'].
team_1_name
Professor Teabag       1
Hareem Kunt            2
Salmon Sisters         3
Hike School            4
Bulgogi                5
Champ                  6
Bottom Bitches         7
Burrito House          8
Pepper Brooks          9
1503 Sequoia Trail    10
Name: optimal_final_rank, dtype: int64
team_1_name
Professor Teabag       1
Hike School            2
Bulgogi                3
Hareem Kunt            4
Champ                  5
Salmon Sisters         6
Bottom Bitches         7
Burrito House          8
Pepper Brooks          9
1503 Sequoia Trail    10
Name: optimal_final_rank, dtype: int64
Week 9 Hareem Kunt got ['Phillip Lindsay', 'Matthew Stafford']. Hike School got ['Nick Chubb', 'Jacoby Brissett'].
team_1_name
Professor Teabag       1
Hareem Kunt            2
Salmon Sisters         3
Hike School            4
Bulgogi                5
Champ                

In [455]:
recordsDf = get_records_optimal(players, matchups, True, "optimal_final_rank").sort_values("final_rank")[["final_rank", "optimal_final_rank"]]
for index, trade in trades.iterrows():
    print(f"Trade {index + 1}: Week {trade['week_enacted']} {trade['trader_team_name']} got {trade['trader_player_names_received']}. {trade['tradee_team_name']} got {trade['tradee_player_names_received']}.")
    playersDfAugmented = get_players_copy_without_trade(players, trade)
    columnName = f"optimal_without_trade_{index}"
    tradeRecords = get_records_optimal(playersDfAugmented, matchups, False, columnName)[columnName]
    recordsDf = recordsDf.join(tradeRecords)
recordsDf

Trade 1: Week 9 Salmon Sisters got ['Jacoby Brissett', 'Darrell Henderson Jr.']. Hike School got ['Sam Darnold', 'Devin Singletary'].
Trade 2: Week 9 Hareem Kunt got ['Phillip Lindsay', 'Matthew Stafford']. Hike School got ['Nick Chubb', 'Jacoby Brissett'].
Trade 3: Week 8 Bottom Bitches got []. Professor Teabag got ['Ezekiel Elliott'].
Trade 4: Week 8 Salmon Sisters got ['Austin Hooper', 'Tyrell Williams']. Bottom Bitches got ['Courtland Sutton', 'Mike Gesicki'].
Trade 5: Week 8 Pepper Brooks got ['Chris Carson', 'Devonta Freeman', 'A.J. Green']. Professor Teabag got ['DJ Moore', 'Christian McCaffrey', 'Matt Breida'].
Trade 6: Week 8 Burrito House got []. Salmon Sisters got ['Tyreek Hill'].
Trade 7: Week 3 Salmon Sisters got ['Dalvin Cook', 'Alexander Mattison']. Bulgogi got ['Joe Mixon', 'Derrick Henry'].


,final_rank,optimal_final_rank,optimal_without_trade_0,optimal_without_trade_1,optimal_without_trade_2,optimal_without_trade_3,optimal_without_trade_4,optimal_without_trade_5,optimal_without_trade_6
team_1_name,,,,,,,,,
Professor Teabag,1,1,1,1,1,1,1,1,1
Hike School,2,4,2,4,4,2,4,4,4
Bulgogi,3,5,3,5,9,9,9,5,5
Salmon Sisters,4,3,6,3,3,3,3,3,3
Hareem Kunt,5,2,4,2,2,4,2,2,2
Champ,6,6,5,6,5,5,6,6,6
Bottom Bitches,7,7,7,7,6,6,7,7,7
Burrito House,8,8,8,8,8,8,8,8,8
Pepper Brooks,9,9,9,9,7,7,5,9,9


In [468]:
text_file = open("trades2019.md", "w")
text_file.write(trades.to_markdown())
text_file.close()

In [469]:
text_file = open("records2019.md", "w")
text_file.write(recordsDf.to_markdown())
text_file.close()

In [432]:
trade = trades.iloc[3]
playersDfAugmented = get_players_copy_without_trade(players, trade)
print(f"{trade['trader_team_name']} got {trade['trader_player_names_received']}. {trade['tradee_team_name']} got {trade['tradee_player_names_received']}.")
get_records_optimal(playersDfAugmented, matchups)

Salmon Sisters got ['Austin Hooper', 'Tyrell Williams']. Bottom Bitches got ['Courtland Sutton', 'Mike Gesicki'].


,wins,losses,regular_season_points,optimal_wins,optimal_losses,optimal_regular_season_points,division_id,optimal_rank,optimal_final_rank
team_1_name,,,,,,,,,
1503 Sequoia Trail,4,9,1688.52,5,8,1882.32,1,9,10
Bottom Bitches,4,9,1621.31,6,7,1808.22,1,6,6
Bulgogi,7,6,1878.42,5,8,2024.88,1,7,9
Burrito House,3,10,1580.91,2,11,1745.24,1,10,8
Champ,6,7,1743.66,9,4,1926.59,2,5,5
Hareem Kunt,7,6,1781.85,9,4,1962.88,2,4,4
Hike School,8,5,1818.43,7,6,2079.66,1,2,2
Pepper Brooks,5,8,1636.13,5,8,1885.77,2,8,7
Professor Teabag,11,2,1982.57,10,3,2208.86,2,1,1


In [426]:
get_records_optimal(players, matchups)

,wins,losses,regular_season_points,optimal_wins,optimal_losses,optimal_regular_season_points,division_id,optimal_rank,optimal_final_rank
team_1_name,,,,,,,,,
1503 Sequoia Trail,4,9,1688.52,5,8,1882.32,1,8,10
Bottom Bitches,4,9,1621.31,5,8,1823.81,1,9,7
Bulgogi,7,6,1878.42,5,8,2024.88,1,6,5
Burrito House,3,10,1580.91,2,11,1745.24,1,10,8
Champ,6,7,1743.66,9,4,1926.59,2,4,6
Hareem Kunt,7,6,1781.85,9,4,1962.88,2,3,2
Hike School,8,5,1818.43,7,6,2079.66,1,2,4
Pepper Brooks,5,8,1636.13,5,8,1885.77,2,7,9
Professor Teabag,11,2,1982.57,10,3,2208.86,2,1,1


In [387]:
get_records_optimal(playersCopy, matchups).sort_values("optimal_final_rank")["optimal_final_rank"]

team_1_name
Professor Teabag       1
Hareem Kunt            2
Salmon Sisters         3
Hike School            4
Bulgogi                5
Champ                  6
Bottom Bitches         7
Burrito House          8
Pepper Brooks          9
1503 Sequoia Trail    10
Name: optimal_final_rank, dtype: int64